<a href="https://colab.research.google.com/github/Soyeon250/DataAnalysis/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

예시 코드 원본

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# MNIST 데이터 셋 로드
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# 데이터 로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear (28*28, 512)
        self.fc2 = nn.Linear (512, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion (output, target)
        loss.backward()
        optimizer.step()
    print ('Epoch %d Loss: %4f' % (epoch+1, loss.item()))

# 테스트
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        # target = torch.eye(10)[target]
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy: {}%'.format(100 * correct / total))

Epoch 1 Loss: 0.495167
Epoch 2 Loss: 0.326656
Epoch 3 Loss: 0.323552
Epoch 4 Loss: 0.477345
Epoch 5 Loss: 0.281959
Epoch 6 Loss: 0.344013
Epoch 7 Loss: 0.249688
Epoch 8 Loss: 0.091747
Epoch 9 Loss: 0.080903
Epoch 10 Loss: 0.306817
Accuracy: 93.16%


LeakyReLU, MSELoss, Adam

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

# MNIST 데이터 셋 로드
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# 데이터 로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 모델 정의 (활성함수를 LeakyReLU로 변경, 은닉층 추가)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear (28*28, 512)
        self.fc2 = nn.Linear (512, 256) # 은닉층 추가
        self.fc3 = nn.Linear (256, 10)
        self.dropout = nn.Dropout(p=0.5) # 드롭아웃 추가(과적합 방지)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.leaky_relu(self.fc1(x)) # LeakyRelu로 교체
        x = self.dropout(x) # 드롭아웃 적용
        x = F.leaky_relu(self.fc2(x)) # 새로운 은닉층 활성함수 통과
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = Net()
criterion = nn.MSELoss() # 손실함수 MSELoss로 변경
optimizer = optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 Adam으로 변경

# 학습
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=10).float() # 코드 추가(손실함수)
        loss = criterion(output, target_one_hot) # loss 계산 방식 변경
        loss.backward()
        optimizer.step()
    print ('Epoch %d Loss: %4f' % (epoch+1, loss.item()))

# 테스트
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        # target = torch.eye(10)[target]
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy: {}%'.format(100 * correct / total))

Epoch 1 Loss: 0.007102
Epoch 2 Loss: 0.004620
Epoch 3 Loss: 0.012013
Epoch 4 Loss: 0.004023
Epoch 5 Loss: 0.002510
Epoch 6 Loss: 0.001774
Epoch 7 Loss: 0.003607
Epoch 8 Loss: 0.002201
Epoch 9 Loss: 0.003135
Epoch 10 Loss: 0.001357
Accuracy: 97.09%


Tanh, RMSprop


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# MNIST 데이터 셋 로드
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# 데이터 로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 모델 정의 (활성함수를 Tanh로 변경, 은닉층 추가)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear (28*28, 512)
        self.fc2 = nn.Linear (512, 256) # 은닉층 추가
        self.fc3 = nn.Linear (256, 10)
        self.dropout = nn.Dropout(p=0.5) # 드롭아웃 추가(과적합 방지)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.tanh(self.fc1(x)) # Tanh로 교체
        x = self.dropout(x) # 드롭아웃 적용
        x = torch.tanh(self.fc2(x)) # 새로운 은닉층 활성함수 통과
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = Net()
criterion = nn.MSELoss() # 손실함수 MSELoss로 변경
optimizer = optim.RMSprop(model.parameters(), lr=0.001) # 옵티마이저 RMSprop로 변경

# 학습
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=10).float() # 코드 추가(손실함수)
        loss = criterion(output, target_one_hot) # loss 계산 방식 변경
        loss.backward()
        optimizer.step()
    print ('Epoch %d Loss: %4f' % (epoch+1, loss.item()))

# 테스트
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        # target = torch.eye(10)[target]
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy: {}%'.format(100 * correct / total))

Epoch 1 Loss: 0.022202
Epoch 2 Loss: 0.023279
Epoch 3 Loss: 0.015636
Epoch 4 Loss: 0.035988
Epoch 5 Loss: 0.012738
Epoch 6 Loss: 0.015952
Epoch 7 Loss: 0.034525
Epoch 8 Loss: 0.023279
Epoch 9 Loss: 0.030445
Epoch 10 Loss: 0.022818
Accuracy: 95.49%
